In [ ]:
from google.colab import drive 
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
claim_data_dir = "drive/MyDrive/claim-data/claim-data/"
claim_filenames = [claim_data_dir + filename for filename in os.listdir(claim_data_dir)]
print(claim_filenames[0])
print(claim_filenames[1])
print(claim_filenames[2])
print(claim_filenames[3])



drive/MyDrive/claim-data/claim-data/test_labels.json
drive/MyDrive/claim-data/claim-data/README.md
drive/MyDrive/claim-data/claim-data/validation_labels.json
drive/MyDrive/claim-data/claim-data/train_labels.json


In [ ]:
import pandas as pd
claim_train_data= pd.read_json(claim_filenames[3], lines=True)
claim_test_data= pd.read_json(claim_filenames[0], lines=True)
claim_val_data= pd.read_json(claim_filenames[2], lines=True)

def sentences_label(claim_train_data):
    ID =[]
    sentences=[]
    labels=[]
    list_df=[]
    claim_sentences=claim_train_data['sentences']
    claim_labels= claim_train_data['labels']
    paper_id = claim_train_data['paper_id']
    for i in range(len(claim_sentences)):
        for j in range(len(claim_sentences[i])):
            sentences.append(claim_sentences[i][j])
            labels.append(int(claim_labels[i][j]))
            ID.append(paper_id[i])
            list_df.append([claim_sentences[i][j],int(claim_labels[i][j])])
    df = pd.DataFrame(list_df, columns=['text', 'labels'])
    return df, sentences,labels
    
train_df, train_sentences,train_labels = sentences_label(claim_train_data)
eval_df,eval_sentences,eval_labels = sentences_label(claim_val_data)
test_df,test_sentences,test_labels = sentences_label(claim_test_data)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
!pip install tensorflow_text
import tensorflow_text as text

     |████████████████████████████████| 4.9 MB 4.2 MB/s 


In [ ]:
import numpy as np 
import pandas as pd

!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece
!pip install keras
import tokenization
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [ ]:
label = preprocessing.LabelEncoder()
y = label.fit_transform(train_df['labels'])
y = to_categorical(y)
print(y[:5])

!pip install tensorflow-addons

import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
     |████████████████████████████████| 1.1 MB 4.3 MB/s 


In [ ]:
m_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(m_url, trainable=True)


vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len-len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

max_len=120
def create_encoder(bert_layer, max_len=120):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids]) 
    #out = sequence_output[:, 0, :]
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output], name="Bert-encoder")
    return model

encoder = create_encoder(bert_layer, max_len=120)
encoder.summary()

Model: "Bert-encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 120)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 120)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 120)]        0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 120, 768)]                'input_mask[0][0]',  

In [ ]:
def build_model(encoder, bert_layer,max_len=120,trainable=True):
    for layer in encoder.layers:
        layer.trainable = trainable
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    pooled_output, sequence_output = encoder([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    lay = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    #lay = tf.keras.layers.Dense(32, activation='relu')(lay)
    #lay = tf.keras.layers.Dropout(0.2)(lay)
    out = tf.keras.layers.Dense(2, activation='softmax')(lay)   
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
train_df=train_df.sample(frac = 1)
y = label.fit_transform(train_df['labels'])
y = to_categorical(y)
train_df

,text,labels
2640,Metabolites of compound 2 formed in vitro by h...,0
3260,"In summary, the sonication technique improves ...",1
4265,We showed previously that the level of Mina ge...,0
4600,The repeating unit of the polysaccharide (PS-I...,0
1313,"We expressed a glucansucrase, DsrI, from Leuco...",0
...,...,...
2602,To examine the neural correlates of contextual...,0
5078,"From 2010 to 2012, 22 patients received cyclop...",0
2388,Two reaction channels were proposed in the pyr...,0
3121,Cancer Res; 76(10); 2857-62.,0


In [ ]:
max_len = 120
train_input = bert_encode(train_df.text.values, tokenizer, max_len=max_len)
test_input = bert_encode(test_df.text.values, tokenizer, max_len=max_len)
eval_input = bert_encode(eval_df.text.values, tokenizer, max_len=max_len)
train_labels = y

In [ ]:
labels = label.classes_
print(labels)

[0 1]


In [ ]:
model = build_model(encoder,bert_layer, max_len=max_len)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 120)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 120)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 120)]        0           []                               
                                                                                                  
 Bert-encoder (Functional)      [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 120, 768)]                'input_mask[0][0]',       

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
tf.config.run_functions_eagerly(True)
train_sh = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=10,
    batch_size=8
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/10
583/583 [==============================] - 341s 574ms/step - loss: 0.2983 - accuracy: 0.8776 - val_loss: 0.2759 - val_accuracy: 0.8747
Epoch 2/10
583/583 [==============================] - 330s 566ms/step - loss: 0.1781 - accuracy: 0.9272 - val_loss: 0.2702 - val_accuracy: 0.8996
Epoch 3/10
583/583 [==============================] - 313s 537ms/step - loss: 0.0946 - accuracy: 0.9650 - val_loss: 0.3149 - val_accuracy: 0.8910
Epoch 4/10
583/583 [==============================] - 332s 570ms/step - loss: 0.0532 - accuracy: 0.9783 - val_loss: 0.4574 - val_accuracy: 0.8996
Epoch 5/10
583/583 [==============================] - 334s 573ms/step - loss: 0.0270 - accuracy: 0.9914 - val_loss: 0.5013 - val_accuracy: 0.8867
Epoch 6/10
583/583 [==============================] - 315s 540ms/step - loss: 0.0214 - accuracy: 0.9929 - val_loss: 0.6693 - val_accuracy: 0.8884
Epoch 7/10
583/583 [==============================] - 316s 542ms/step - loss: 0.0147 - accuracy: 0.9961 - val_loss: 0.5597 -

In [ ]:
X_val,  y_val = eval_df['text'], eval_df['labels']
X_test,  y_test = test_df['text'], test_df['labels']

In [ ]:
Y_pred = np.argmax(model.predict(test_input), axis = 1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, Y_pred))

Y_pred_val = np.argmax(model.predict(eval_input), axis = 1)
print(classification_report(y_val, Y_pred_val))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      2362
           1       0.79      0.66      0.72       571

    accuracy                           0.90      2933
   macro avg       0.86      0.81      0.83      2933
weighted avg       0.90      0.90      0.90      2933



/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


              precision    recall  f1-score   support

           0       0.92      0.96      0.94      2361
           1       0.80      0.66      0.72       585

    accuracy                           0.90      2946
   macro avg       0.86      0.81      0.83      2946
weighted avg       0.90      0.90      0.90      2946

